In [2]:
import numpy as np
import pandas as pd
import httpx
import asyncio
import ujson as json
from datetime import datetime
import os
from typing import Dict, TypeAlias

from script import FedInvestFetcher
from RL_BondPricer import RL_BondPricer

import multiprocessing as mp
from functools import partial 


JSON: TypeAlias = dict[str, "JSON"] | list["JSON"] | str | int | float | bool | None

import nest_asyncio
nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

In [3]:
def calculate_yields(row, as_of_date):
    offer_yield = RL_BondPricer.bond_price_to_ytm(
        type=row["security_type"],
        issue_date=row["issue_date"],
        maturity_date=row["maturity_date"],
        as_of=as_of_date,
        coupon=float(row["int_rate"]) / 100,
        price=row["offer_price"],
    )
    bid_yield = RL_BondPricer.bond_price_to_ytm(
        type=row["security_type"],
        issue_date=row["issue_date"],
        maturity_date=row["maturity_date"],
        as_of=as_of_date,
        coupon=float(row["int_rate"]) / 100,
        price=row["bid_price"],
    )
    eod_yield = RL_BondPricer.bond_price_to_ytm(
        type=row["security_type"],
        issue_date=row["issue_date"],
        maturity_date=row["maturity_date"],
        as_of=as_of_date,
        coupon=float(row["int_rate"]) / 100,
        price=row["eod_price"],
    )

    return offer_yield, bid_yield, eod_yield

In [4]:
def runner(dates):
    async def build_tasks(client: httpx.AsyncClient, dates):
        tasks = await FedInvestFetcher(
            use_ust_issue_date=True, error_verbose=True
        )._build_fetch_tasks_historical_cusip_prices(
            client=client, dates=dates, max_concurrent_tasks=5
        )
        return await asyncio.gather(*tasks)

    async def run_fetch_all(dates):
        limits = httpx.Limits(max_connections=5)
        async with httpx.AsyncClient(limits=limits) as client:
            all_data = await build_tasks(client=client, dates=dates)
            return all_data

    results = asyncio.run(run_fetch_all(dates=dates))
    return dict(results)

In [5]:
raw_auctions_df = FedInvestFetcher(use_ust_issue_date=True, error_verbose=True).get_auctions_df()
raw_auctions_df["issue_date"] = pd.to_datetime(raw_auctions_df["issue_date"])
raw_auctions_df["maturity_date"] = pd.to_datetime(
    raw_auctions_df["maturity_date"]
)
raw_auctions_df["auction_date"] = pd.to_datetime(
    raw_auctions_df["auction_date"]
)
raw_auctions_df.loc[
    raw_auctions_df["original_security_term"].str.contains(
        "29-Year", case=False, na=False
    ),
    "original_security_term",
] = "30-Year"
raw_auctions_df.loc[
    raw_auctions_df["original_security_term"].str.contains(
        "30-", case=False, na=False
    ),
    "original_security_term",
] = "30-Year"

raw_auctions_df = raw_auctions_df[
    (raw_auctions_df["security_type"] == "Bill")
    | (raw_auctions_df["security_type"] == "Note")
    | (raw_auctions_df["security_type"] == "Bond")
]
raw_auctions_df = raw_auctions_df.drop(
    raw_auctions_df[
        (raw_auctions_df["security_type"] == "Bill")
        & (
            raw_auctions_df["original_security_term"]
            != raw_auctions_df["security_term"]
        )
    ].index
)
raw_auctions_df

2024-08-18 14:04:06,008 - script - DEBUG - UST Auctions - Number of Links to Fetch: 2
2024-08-18 14:04:06,010 - script - DEBUG - UST Auctions - Links: ['https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query?page[number]=1&page[size]=10000', 'https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query?page[number]=2&page[size]=10000']


,cusip,security_type,auction_date,issue_date,maturity_date,price_per100,allocation_pctage,avg_med_yield,bid_to_cover_ratio,comp_accepted,...,security_term,original_security_term,security_term_week_year,primary_dealer_accepted,primary_dealer_tendered,reopening,total_accepted,total_tendered,treas_retail_accepted,treas_retail_tenders_accepted
193,912797MP9,Bill,2024-08-14,2024-08-20,2024-12-17,98.353833,18.380000,null,3.340000,59323080000,...,17-Week,17-Week,17-Week,17856070000,121650000000,No,60191055400,200587675400,485699200,Yes
191,912797MK0,Bill,2024-08-12,2024-08-15,2025-02-13,97.575861,36.280000,null,2.730000,67602581000,...,26-Week,26-Week,26-Week,25933500000,131875000000,No,76404564800,197600562800,1107659100,Yes
187,912810UC0,Bond,2024-08-08,2024-08-15,2054-08-15,98.928757,14.970000,4.225000,2.310000,24941455500,...,30-Year,30-Year,30-Year,4784470000,32199000000,No,29754413000,62379781500,21843600,Yes
188,91282CLF6,Note,2024-08-07,2024-08-15,2034-08-15,99.303721,91.610000,3.888000,2.320000,41874806900,...,10-Year,10-Year,10-Year,7486140000,54904700000,No,49987416200,105290699300,73631000,Yes
189,91282CLG4,Note,2024-08-06,2024-08-15,2027-08-15,99.831417,92.120000,3.750000,2.550000,57745565000,...,3-Year,3-Year,3-Year,8866300000,82526000000,No,69030182700,159002762700,109350200,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,9127934W3,Bill,1980-01-14,1980-01-17,1980-07-17,null,86.000000,null,null,null,...,26-Week,26-Week,26-Week,null,null,No,3201655000,6288420000,null,null
6,9127934V5,Bill,1980-01-07,1980-01-10,1980-07-10,null,94.000000,null,null,null,...,26-Week,26-Week,26-Week,null,null,No,3201390000,5837550000,null,null
7,912810CL0,Bond,1980-01-03,1980-01-10,1995-02-15,null,45.000000,10.600000,null,null,...,15-Year 1-Month,15-Year 1-Month,15-Year,null,null,No,1501000000,3724000000,null,null
3,9127934U7,Bill,1979-12-28,1980-01-03,1980-07-03,null,94.000000,null,null,null,...,26-Week,26-Week,26-Week,null,null,No,3348725000,5954350000,null,null


In [6]:
def get_business_days_groups(start_date: datetime, end_date: datetime, group_size=3):
    date_range = pd.date_range(start=start_date, end=end_date, freq="B")
    business_day_groups = [
        [bday.to_pydatetime() for bday in date_range[i : i + group_size].tolist()]
        for i in range(0, len(date_range), group_size)
    ]
    return business_day_groups


start_date = datetime(2024, 8, 14)
end_date = datetime(2024, 8, 16)
weeks = get_business_days_groups(start_date, end_date, group_size=5)

In [7]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [9]:
for week in weeks:
    dict_df: Dict[datetime, pd.DataFrame] = runner(dates=week)
    output_directory = r"C:\Users\chris\CUSIP-Set"
    for key, df in dict_df.items():
        try:
            cusip_ref_df = raw_auctions_df[
                raw_auctions_df["cusip"].isin(df["cusip"].to_list())
            ][["cusip", "security_type", "issue_date", "maturity_date", "int_rate"]]
            merged_df = pd.merge(left=df, right=cusip_ref_df, on=["cusip"])
            merged_df = merged_df.replace("null", np.nan)
            
            calculate_yields_partial = partial(
                calculate_yields, as_of_date=key
            )
            # with mp.Pool(mp.cpu_count()) as pool:
            #     results = pool.map(
            #         calculate_yields_partial, [row for _, row in merged_df.iterrows()]
            #     )
            # offer_yields, bid_yields, eod_yields = zip(*results)
            # merged_df["offer_yield"] = offer_yields
            # merged_df["bid_yield"] = bid_yields
            # merged_df["eod_yield"] = eod_yields
            
            merged_df["eod_yield"] = merged_df.apply(
                lambda row: RL_BondPricer.bond_price_to_ytm(
                    type=row["security_type"],
                    issue_date=row["issue_date"],
                    maturity_date=row["maturity_date"],
                    as_of=key,
                    coupon=float(row["int_rate"]) / 100,
                    price=row["eod_price"],
                ),
                axis=1,
            )
            merged_df["bid_yield"] = merged_df.apply(
                lambda row: RL_BondPricer.bond_price_to_ytm(
                    type=row["security_type"],
                    issue_date=row["issue_date"],
                    maturity_date=row["maturity_date"],
                    as_of=key,
                    coupon=float(row["int_rate"]) / 100,
                    price=row["bid_price"],
                ),
                axis=1,
            )
            merged_df["offer_yield"] = merged_df.apply(
                lambda row: RL_BondPricer.bond_price_to_ytm(
                    type=row["security_type"],
                    issue_date=row["issue_date"],
                    maturity_date=row["maturity_date"],
                    as_of=key,
                    coupon=float(row["int_rate"]) / 100,
                    price=row["offer_price"],
                ),
                axis=1,
            )
            merged_df["mid_price"] = (
                merged_df["offer_price"] + merged_df["bid_price"]
            ) / 2
            merged_df["mid_yield"] = (
                merged_df["offer_yield"] + merged_df["bid_yield"]
            ) / 2
            
            merged_df = merged_df[["cusip", "bid_price", "offer_price", "mid_price", "eod_price", "bid_yield", "offer_yield", "eod_yield", "eod_yield"]]
            records = merged_df.to_dict(orient='records')

            json_structure = {"data": records}
            file_name = f"{key.strftime("%Y-%m-%d")}.json"
            file_path = os.path.join(output_directory, file_name)

            with open(file_path, 'w') as json_file:
                json.dump(json_structure, json_file, indent=4)
        
        except Exception as e:
            print(bcolors.FAIL + f"FAILED ON {key} - {str(e)}" + bcolors.ENDC)

2024-08-18 14:05:01,972 - script - DEBUG - UST Prices - 2024-08-14 00:00:00 Payload: {'priceDate.month': 8, 'priceDate.day': 14, 'priceDate.year': 2024, 'submit': 'Show Prices'}
2024-08-18 14:05:01,973 - script - DEBUG - UST Prices - 2024-08-15 00:00:00 Payload: {'priceDate.month': 8, 'priceDate.day': 15, 'priceDate.year': 2024, 'submit': 'Show Prices'}
2024-08-18 14:05:01,973 - script - DEBUG - UST Prices - 2024-08-16 00:00:00 Payload: {'priceDate.month': 8, 'priceDate.day': 16, 'priceDate.year': 2024, 'submit': 'Show Prices'}
2024-08-18 14:05:02,262 - script - DEBUG - UST Prices - 2024-08-14 00:00:00 Redirecting to https://savingsbonds.gov/GA-FI/FedInvest/securityPriceDetail;jsessionid=FB6D3637A7DFB2D8E4DDDA48C55B43A5
2024-08-18 14:05:02,283 - script - DEBUG - UST Prices - 2024-08-15 00:00:00 Redirecting to https://savingsbonds.gov/GA-FI/FedInvest/securityPriceDetail;jsessionid=CBE9A8355CA112E7D3C070788144F552
2024-08-18 14:05:02,285 - script - DEBUG - UST Prices - 2024-08-16 00:00:0